In [134]:
import pandas as pd
from io import StringIO

data_description = "The Dataset offers valuable insights into consumer shopping behavior and purchasing patterns."
data=pd.read_csv('tabular_data.csv')
sample_data=data.sample(n=10)
sample_data

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Review Rating,Subscription Status,Payment Method,Discount Applied
2471,2472,30.0,Male,Sandals,Footwear,83,4.3,No,NaN,No
2014,2015,NaN,Male,Skirt,Clothing,26,3.9,No,NaN,No
2791,2792,43.0,Female,Gloves,Accessories,60,2.8,No,Credit Card,No
2997,2998,41.0,Female,NaN,Clothing,40,NaN,No,NaN,No
620,621,33.0,Male,Backpack,Accessories,89,3.7,Yes,PayPal,Yes
630,631,41.0,Male,Jewelry,Accessories,94,4.3,Yes,NaN,Yes
2267,2268,32.0,NaN,Gloves,Accessories,49,3.8,No,NaN,No
1787,1788,NaN,Male,Sweater,Clothing,38,NaN,No,Bank Transfer,No
1951,1952,33.0,NaN,Gloves,Accessories,53,NaN,No,Bank Transfer,No
2425,2426,58.0,Male,NaN,Footwear,49,3.0,No,Venmo,No


In [135]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3900 entries, 0 to 3899
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Customer ID            3900 non-null   int64  
 1   Age                    3510 non-null   object 
 2   Gender                 3120 non-null   object 
 3   Item Purchased         3315 non-null   object 
 4   Category               3900 non-null   object 
 5   Purchase Amount (USD)  3900 non-null   int64  
 6   Review Rating          2730 non-null   float64
 7   Subscription Status    3900 non-null   object 
 8   Payment Method         2925 non-null   object 
 9   Discount Applied       3900 non-null   object 
dtypes: float64(1), int64(2), object(7)
memory usage: 304.8+ KB


In [136]:
data.head()

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Review Rating,Subscription Status,Payment Method,Discount Applied
0,1,NaN,Male,Blouse,Clothing,53,3.1,Yes,Credit Card,Yes
1,2,19.0,Male,NaN,Clothing,64,NaN,Yes,Bank Transfer,Yes
2,3,NaN,NaN,Jeans,Clothing,73,NaN,Yes,Cash,Yes
3,4,21.0,Male,Sandals,45,90,NaN,Yes,PayPal,Yes
4,5,45.0,NaN,Blouse,Clothing,49,2.7,Yes,Cash,Yes


In [137]:
csv_text = StringIO()
sample_data.to_csv(csv_text, index=False, header=True)

In [138]:
csv_content = csv_text.getvalue()
print(csv_content)

Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Review Rating,Subscription Status,Payment Method,Discount Applied
2472,30.0,Male,Sandals,Footwear,83,4.3,No,,No
2015,,Male,Skirt,Clothing,26,3.9,No,,No
2792,43.0,Female,Gloves,Accessories,60,2.8,No,Credit Card,No
2998,41.0,Female,,Clothing,40,,No,,No
621,33.0,Male,Backpack,Accessories,89,3.7,Yes,PayPal,Yes
631,41.0,Male,Jewelry,Accessories,94,4.3,Yes,,Yes
2268,32.0,,Gloves,Accessories,49,3.8,No,,No
1788,,Male,Sweater,Clothing,38,,No,Bank Transfer,No
1952,33.0,,Gloves,Accessories,53,,No,Bank Transfer,No
2426,58.0,Male,,Footwear,49,3.0,No,Venmo,No



In [139]:
from dotenv import load_dotenv
load_dotenv()
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI

model=ChatGoogleGenerativeAI(model="gemini-1.5-flash")
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(
        content='''You are a part of an application you have to perform some tasks.

Context: It is an application which gives the quality score for tabular data on the basis of issues listed down.

issues = { 
    
    "column-issues": {
       column1Name:{
        "Duplicated Values": "confidence here",
        "Outliers": "confidence here",
        }
       column2Name:..........So On.
    },
    "cell-issues": {
        column1Name:{
        "Missing Values": "confidence here"
        "Inconsistency": "confidence here"
        }
       column2Name:..........So On.
    }
}

Confidence: It is basically the value between 0 to 1. And it states that how important the issue is with respect to a dataset. Means If confidence is close to 1 then issue is very important to user, and he don't want to ignore this issue. Means the weightage of issue will be more.

Description of issues:

Column-Level Issues:
1) Duplicated Values: Columns expected to have unique values (e.g., identifiers) but contain duplicates.
2) Outliers: Extreme values that are statistically distant from most of the values in the column.

Cell-Level Issues:
1) Missing Values: Null or empty values where data is expected (e.g., in non-nullable columns).
2) Inconsistency: Values that don't conform to expected formats, patterns, or domain constraints. For example, a date column containing a non-date value or a numerical column containing text.

Now these issues will have the different confidence score with respect to different columns of different dataset.

Task: You have to analyze the given dataset description and the dataset and its columns/features, understand the importance of issues for the given dataset and fill the below format with the confidence score keeping in mind the below given rules for each issues. 
Rules:
    Rules for Duplicate value issue:
    Below given features and similar to those will have duplicate value confidence 1
        -IDs, Primary Keys, Unique Keys, Identifiers.
        -Examples : user_id, order_id, transaction_id, email, but not limited to this.
        -These features are crucial for maintaining the uniqueness of records in a dataset.
    Below given features and similar to those will have duplicate value confidence 0
        -Categorical Features with No Uniqueness Requirement
        -Examples : gender, status, education_level, region, but not limited to this.
        -These features are used for classification and do not require unique values.
    For other features, assign confidence between 0 and 1 based on the importance of Duplicate value issue for that feature. So don't assign 0 and 1 only.

    Rules for Missing value issue:
    Below given features and similar to those will have missing value confidence 1:
        - Features that cannot have null values due to their critical nature in the dataset.
        - Examples: IDs, Primary Keys, mandatory fields like created_at, order_id, transaction_id, but not limited to this.
        - These are essential features where missing values would affect data integrity.
    Below given features and similar to those will have missing value confidence 0:
        - Features where missing data is acceptable or does not heavily impact analysis.
        - Examples: comments, feedback, review_text, secondary_contact_number, but not limited to this.
        - These are optional or free-text fields where null values are expected or permissible.
    For other features, assign confidence between 0 and 1 based on the importance of missing values for that feature. So don't assign 0 and 1 only.

    Rules for Outlier issue:
    Below given features and similar to those will have outlier confidence 1:
        - Numeric Features with Defined Ranges, Sensitive Data.
        - Examples: salary, age, percentage, transaction_amount, but not limited to this.
        - These features must remain within specific thresholds, and outliers could distort analysis or represent errors.
    Below given features and similar to those will have outlier confidence 0:
        - Features where outliers are either expected or do not significantly impact analysis.
        - Examples: gender, education_level, region, feedback, comments, but not limited to this.
        - Categorical or free-text fields where there is no concept of outliers.
    For other features, assign confidence between 0 and 1 based on the importance of outlier for that feature. So don't assign 0 and 1 only.

    Rules for Inconsistency Issue:
    Below given features and similar to those will have inconsistency issue confidence 1.
        - Features where data must adhere strictly to specific datatype, formats, patterns, or domains.
        - Examples: IDs, Primary Keys, Unique Keys, Identifiers, date_of_birth (must follow date format), email (must follow a valid email pattern), phone_number (must follow a valid number pattern), but not limited to this.
        - These features have strict requirements for data integrity, and inconsistent values could represent significant errors.
    Below given features and similar to those will have inconsistency issue confidence 0.
        - Features where there is no strict requirement for format or pattern adherence.
        - Examples: comments, feedback, review_text.
        - These are free-text or optional fields where data variation is expected, and inconsistency does not significantly affect the analysis.
    For other features, assign confidence between 0 and 1 based on the importance of data consistency for that feature. So don't assign 0 and 1 only.

Output Format:

issues-confidence = { 
    
    "column-issues": {
       column1Name:{ 
        "Duplicated Values": "confidence here",
        "Outliers": "confidence here",
        }
       column2Name:..........So On.
    },
    "cell-issues": {
        column1Name:{ 
        "Missing Values": "confidence here"
        "Inconsistency": "confidence here"
        }
       column2Name:..........So On.
    }
}

Output: 
Fill the format given above and return it in json. Don't write anything else.

'''
    ),
    HumanMessage(
        content=f"""
        Data Description: {data_description}
        Data: {csv_content}
        """
    )
]

# Define a chat model and invoke it with the messages
result1=model.invoke(messages)
result2=model.invoke(messages)
result3=model.invoke(messages)
result4=model.invoke(messages)

In [140]:
import json

def LLM_result_to_JSON(result):
    cleaned_content = result.content.replace('```json', '').replace('```', '').strip()
    issue_confidence = json.loads(cleaned_content)
    return issue_confidence

issue_confidence1=LLM_result_to_JSON(result1)
issue_confidence2=LLM_result_to_JSON(result2)
issue_confidence3=LLM_result_to_JSON(result3)
issue_confidence4=LLM_result_to_JSON(result4)

responses=[issue_confidence1,issue_confidence2,issue_confidence3,issue_confidence4]

In [141]:
def average_responses(responses):
    result = {}

    num_responses = len(responses)

    for response in responses:
        for issue_type, columns in response.items():
            if issue_type not in result:
                result[issue_type] = {}
            for column, metrics in columns.items():
                if column not in result[issue_type]:
                    result[issue_type][column] = {}
                for metric, value in metrics.items():
                    if metric not in result[issue_type][column]:
                        result[issue_type][column][metric] = 0
                    result[issue_type][column][metric] += float(value)

    # Divide summed values by number of responses to get the average
    for issue_type, columns in result.items():
        for column, metrics in columns.items():
            for metric in metrics:
                result[issue_type][column][metric] /= num_responses

    return result

# Calculate the averaged response
issue_confidence = average_responses(responses)
issue_confidence

{'column-issues': {'Customer ID': {'Duplicated Values': 1.0, 'Outliers': 0.0},
  'Age': {'Duplicated Values': 0.5, 'Outliers': 0.8},
  'Gender': {'Duplicated Values': 0.0, 'Outliers': 0.0},
  'Item Purchased': {'Duplicated Values': 0.1, 'Outliers': 0.0},
  'Category': {'Duplicated Values': 0.0, 'Outliers': 0.0},
  'Purchase Amount (USD)': {'Duplicated Values': 0.175,
   'Outliers': 0.7750000000000001},
  'Review Rating': {'Duplicated Values': 0.2, 'Outliers': 0.55},
  'Subscription Status': {'Duplicated Values': 0.0, 'Outliers': 0.0},
  'Payment Method': {'Duplicated Values': 0.025, 'Outliers': 0.0},
  'Discount Applied': {'Duplicated Values': 0.0, 'Outliers': 0.0}},
 'cell-issues': {'Customer ID': {'Missing Values': 0.0, 'Inconsistency': 1.0},
  'Age': {'Missing Values': 0.675, 'Inconsistency': 0.3},
  'Gender': {'Missing Values': 0.525, 'Inconsistency': 0.05},
  'Item Purchased': {'Missing Values': 0.15000000000000002,
   'Inconsistency': 0.1},
  'Category': {'Missing Values': 0.25, 

In [142]:
from dateutil import parser

def is_valid_string(val):
        if isinstance(val, str) and not pd.isnull(val):
            try:
                float(val)  # Try to convert to float
                return False  # If it's a float, return False
            except ValueError:
                try:
            # Try to parse the string as any valid datetime format
                    parser.parse(val)
                    return False  # It's a valid datetime, so not a valid string
                except (ValueError, TypeError):
                    return True  # It's not a datetime, so it's a valid string
        return False

def is_valid_numeric(val):
    try:
        float(val)  # Try to convert the value to a float
        return True  # If conversion succeeds, return True
    except (ValueError, TypeError):
        return False  # If conversion fails, it's not a valid numeric value

def is_valid_datetime(val):
    try:
        float(val)
        return False
    except (ValueError, TypeError):
        try:
        # Attempt to parse the value as a datetime
            parser.parse(val)
            return True  # If parsing succeeds, it's a valid datetime
        except (ValueError, TypeError):
            return False  # If parsing fails, it's not a valid datetime

def find_column_data_type(column_name):
    num_numeric = data[column_name].apply(is_valid_numeric).sum()
    num_datetime = data[column_name].apply(is_valid_datetime).sum()
    num_string = data[column_name].apply(is_valid_string).sum()
    
    # Infer type based on the majority of data
    if num_numeric > num_string and num_numeric > num_datetime:
        return 'numeric'
    elif num_datetime > num_string and num_datetime > num_numeric:
        return 'datetime'
    else:
        return 'string'
    
def is_valid_datatype(row_index, column_name, detected_datatype):
    if detected_datatype == 'numeric':
        return is_valid_numeric(data.loc[row_index,column_name])
    elif detected_datatype == 'datetime':
        return is_valid_datetime(data.loc[row_index,column_name])
    else:
        return is_valid_string(data.loc[row_index,column_name])

In [143]:
#functions to find accuracy score corresponding to each issues

import numpy as np

#For column issues

def accuracy_score_correspond_to_duplicated_values(column_name):
    prevalance=(data[column_name].duplicated().sum())/len(data)
    confidence=float(issue_confidence['column-issues'][column_name]['Duplicated Values'])

    return float(1-prevalance*confidence)

def accuracy_score_correspond_to_outliers(column_name):
    if find_column_data_type(column_name)!='numeric':
        return float(1-0)
    
    # Apply the is_valid_numeric function to each element in the column
    column = data[column_name].apply(lambda x: x if is_valid_numeric(x) else np.nan)

    # Convert the valid numeric entries to float for accurate calculations
    column = column.astype(float)

    # Now calculate the lower and upper bounds
    lowerBound = column.mean() - 3 * column.std()
    upperBound = column.mean() + 3 * column.std()

    outlier_flags=column.apply(lambda x:x<lowerBound or x>upperBound)

    prevalance=outlier_flags.mean()
    confidence=float(issue_confidence['column-issues'][column_name]['Outliers'])

    return float(1-prevalance*confidence)

def accuracy_score_correspond_to_class_imbalance(column_name):
    if find_column_data_type(column_name)=='numeric' or find_column_data_type(column_name)=='datetime':
        return float(1-0)
    
    # Get the count of each class in the 'Category' column
    class_counts = data[column_name].value_counts()

    # Find the minimum and maximum counts
    min_count = class_counts.min()
    max_count = class_counts.max()

    prevalance=(max_count-min_count)/(max_count+min_count)

    return float(1-prevalance)


#For cell issues

def accuracy_score_correspond_to_missing_values(row_name,column_name):
    cell_value= data.loc[row_name,column_name]
    confidence=float(issue_confidence['cell-issues'][column_name]['Missing Values'])
    
    if isinstance(cell_value, str):
        # For categorical/string values
        if cell_value.strip() == "":
            return float(1 - confidence)
    elif isinstance(cell_value, (float, int)):
        # For numerical values
        if np.isnan(cell_value):
            return float(1 - confidence)
    elif cell_value is None:
        return float(1 - confidence)
    return 1

def accuracy_score_correspond_to_data_type(row_name,column_name,detected_datatype):
    if not (data[column_name].dtype=='object'):
        return 1
    confidence=float(issue_confidence['cell-issues'][column_name]['Inconsistency'])
    if is_valid_datatype(row_name, column_name, detected_datatype):
        return 1
    return float(1-confidence)

In [144]:
# Create an empty DataFrame to store the final scores
final_cell_score_df = pd.DataFrame(1, index=data.index, columns=data.columns)

issues_scores={}
# Loop over each column in the DataFrame
for column_name in data.columns:
    issues_scores[column_name]={}
    column_detected_dtype= find_column_data_type(column_name)
    column_score_duplicates=0
    column_score_outliers=0
    column_score_class_imbalance=0
    column_score_missing=0
    column_score_datatype=0
    # Dealing column specific issues
    score_duplicates = accuracy_score_correspond_to_duplicated_values(column_name)
    score_outliers = accuracy_score_correspond_to_outliers(column_name)
    score_class_imbalance = accuracy_score_correspond_to_class_imbalance(column_name)
    accuracy_score_column_issue = score_duplicates*score_outliers
   
    # Now, loop over each row in this column
    for row_index in data.index:
        # Dealing cell specific issues
        score_missing = accuracy_score_correspond_to_missing_values(row_index,column_name)
        score_datatype = accuracy_score_correspond_to_data_type(row_index,column_name,column_detected_dtype)
        accuracy_score_cell_issue = score_missing*score_datatype
       
        # Assign the final score for this cell
        final_cell_score_df.at[row_index, column_name] = accuracy_score_column_issue*accuracy_score_cell_issue

        column_score_missing=column_score_missing+score_missing
        column_score_datatype=column_score_datatype+score_datatype
    issues_scores[column_name]['duplicates']=round(score_duplicates*100,2)
    issues_scores[column_name]['outliers']=round(score_outliers*100,2)
    issues_scores[column_name]['imbalance']=round(score_class_imbalance*100,2)
    issues_scores[column_name]['missing']=round((column_score_missing/len(data))*100,2)
    issues_scores[column_name]['datatype']=round((column_score_datatype/len(data))*100,2)

/tmp/ipykernel_2439/2792475133.py:28: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.16484117948717944' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  final_cell_score_df.at[row_index, column_name] = accuracy_score_column_issue*accuracy_score_cell_issue
/tmp/ipykernel_2439/2792475133.py:28: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.45125' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  final_cell_score_df.at[row_index, column_name] = accuracy_score_column_issue*accuracy_score_cell_issue
/tmp/ipykernel_2439/2792475133.py:28: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.9006666666666667' has dtype incompatible with int64, please explicitly cast to a co

In [145]:
final_cell_score_df.head(10)

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Review Rating,Subscription Status,Payment Method,Discount Applied
0,1,0.164841,1.00000,0.900667,1.00,0.828635,0.801385,1,0.975045,1
1,1,0.507204,1.00000,0.689010,1.00,0.828635,0.480831,1,0.975045,1
2,1,0.164841,0.45125,0.900667,1.00,0.828635,0.480831,1,0.975045,1
3,1,0.507204,1.00000,0.900667,0.95,0.828635,0.480831,1,0.975045,1
4,1,0.507204,0.45125,0.900667,1.00,0.828635,0.801385,1,0.975045,1
5,1,0.355043,1.00000,0.900667,1.00,0.828635,0.801385,1,0.975045,1
6,1,0.507204,0.45125,0.900667,1.00,0.828635,0.801385,1,0.975045,1
7,1,0.507204,1.00000,0.900667,1.00,0.828635,0.801385,1,0.975045,1
8,1,0.507204,1.00000,0.900667,1.00,0.828635,0.480831,1,0.975045,1
9,1,0.507204,1.00000,0.900667,1.00,0.828635,0.480831,1,0.975045,1


In [146]:
column_score = final_cell_score_df.mean(numeric_only=True).to_dict()
data_score = final_cell_score_df.values.flatten().mean()

print('Issues_scores',issues_scores)
print('Column Score : ',column_score)
print('Data Score : ',data_score*100)

Issues_scores {'Customer ID': {'duplicates': 100.0, 'outliers': 100.0, 'imbalance': 100.0, 'missing': 100.0, 'datatype': 100.0}, 'Age': {'duplicates': 50.73, 'outliers': 99.98, 'imbalance': 100.0, 'missing': 93.25, 'datatype': 99.98}, 'Gender': {'duplicates': 100.0, 'outliers': 100.0, 'imbalance': 64.62, 'missing': 89.5, 'datatype': 99.0}, 'Item Purchased': {'duplicates': 90.07, 'outliers': 100.0, 'imbalance': 82.31, 'missing': 97.75, 'datatype': 98.5}, 'Category': {'duplicates': 100.0, 'outliers': 100.0, 'imbalance': 0.12, 'missing': 100.0, 'datatype': 100.0}, 'Purchase Amount (USD)': {'duplicates': 82.86, 'outliers': 100.0, 'imbalance': 100.0, 'missing': 100.0, 'datatype': 100.0}, 'Review Rating': {'duplicates': 80.14, 'outliers': 100.0, 'imbalance': 100.0, 'missing': 88.0, 'datatype': 100.0}, 'Subscription Status': {'duplicates': 100.0, 'outliers': 100.0, 'imbalance': 54.0, 'missing': 100.0, 'datatype': 100.0}, 'Payment Method': {'duplicates': 97.5, 'outliers': 100.0, 'imbalance': 9